## Importing libraries

In [1]:
import os, sys, time, datetime, glob, bcolz, random

In [19]:
import keras
import numpy as np
import pandas as pd

In [3]:
from keras import Sequential

In [4]:
import matplotlib.pyplot as plt

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Convolution2D

In [38]:
folder_affectnet = "/media/amogh/Stuff/CMU/datasets/AffectNet/"

In [39]:
folder_train_data = folder_affectnet + 'Manually_Annotated_Images/'

## Generating labels for valence and arousal training

#### Reading and filtering dataframes from csv files

In [117]:
path_validation_csv = folder_affectnet + "Manually_Annotated_file_lists/training_va.csv"
folder_annotated_validation_images = folder_affectnet + "Manually_Annotated_Images/"

Getting image_filename and the corresponding labels for valence and arousal

In [118]:
df_val = pd.read_csv(path_validation_csv,header=None)
df_val.head()

,0,1,2
0,689/737db2483489148d783ef278f43f486c0a97e140fc...,0.785714,-0.055556
1,392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b5532...,-0.017253,0.004313
2,468/21772b68dc8c2a11678c8739eca33adb6ccc658600...,0.174603,0.007937
3,944/06e9ae8d3b240eb68fa60534783eacafce2def60a8...,0.153401,0.038890
4,993/02e06ee5521958b4042dd73abb444220609d96f57b...,0.783972,-0.551684


In [119]:
list_image_paths = df_val.iloc[:,0].values

In [120]:
df_va_validation = df_val.iloc[:,[0,len(df_val.columns)-2,len(df_val.columns)-1]]
df_va_validation.columns = ["image","valence", "arousal"]
print("shape is: ", df_va_validation.shape)
(df_va_validation.head())

shape is:  (414799, 3)


,image,valence,arousal
0,689/737db2483489148d783ef278f43f486c0a97e140fc...,0.785714,-0.055556
1,392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b5532...,-0.017253,0.004313
2,468/21772b68dc8c2a11678c8739eca33adb6ccc658600...,0.174603,0.007937
3,944/06e9ae8d3b240eb68fa60534783eacafce2def60a8...,0.153401,0.038890
4,993/02e06ee5521958b4042dd73abb444220609d96f57b...,0.783972,-0.551684


Converting the dataframe in a dictionary

In [121]:
dict_im_va = df_va_validation.set_index('image').T.to_dict('list')
dict_im_va

{'689/737db2483489148d783ef278f43f486c0a97e140fc4b6b61b84363ca.jpg': [0.785714,
  -0.055555600000000004],
 '392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b55326e9181ee4520f9.jpg': [-0.0172533,
  0.00431332],
 '468/21772b68dc8c2a11678c8739eca33adb6ccc658600e4da2224080603.jpg': [0.174603,
  0.00793651],
 '944/06e9ae8d3b240eb68fa60534783eacafce2def60a86042f9b7d59544.jpg': [0.153401,
  0.0388903],
 '993/02e06ee5521958b4042dd73abb444220609d96f57b1689abbe87c024.jpg': [0.783972,
  -0.5516840000000001],
 '979/f675c6a88cdef99a6d8b0261741217a0319387fcf1571a174f99ac81.jpg': [-0.34126999999999996,
  0.547619],
 '637/94b769d8e880cbbea8eaa1350cb8c094a03d27f9fef44e1f4c0fb2ae.jpg': [-2.0,
  -2.0],
 '997/b81f843f08ce3bb0c48b270dc58d2ab8bf5bea3e2262e50bbcadbec2.jpg': [-0.488145,
  0.831839],
 '358/21a32dd1c1ecd57d3e8964621c911df1c0b3348a4ae5203b4a243230.JPG': [-2.0,
  -2.0],
 '330/60216cf5171a1f18876f178e4ab84db22f2dc0920bdaca819769b032.jpg': [-2.0,
  -2.0],
 '981/5c991a28b5550cf5e1e830d5eed1dc7f80581462d627

In [122]:
len(dict_im_va)

414799

#### Getting numpy array for labels

In [141]:
list_va_labels = [dict_im_va.get(im) if dict_im_va.get(im) is not None else [-2.,-2.] for im in list_filenames_affectnet]
array_va_labels = np.array(list_va_labels)

In [142]:
array_va_labels

array([[-2.       , -2.       ],
       [-2.       , -2.       ],
       [-2.       , -2.       ],
       ...,
       [ 0.626566 ,  0.274123 ],
       [ 0.0714286, -0.460317 ],
       [-0.448585 , -0.150966 ]])

#### Defining function to get labels

In [144]:
def get_va_from_image(image_name, dict_im_va=dict_im_va):
    return np.array(dict_im_va[image_name])

## Reading images from folder and saving convolutional features (for now just training)

In [150]:
def get_data(path, target_size=(224,224), class_mode=None, shuffle=False, batch_size=16):
    gen = ImageDataGenerator()
    batches = gen.flow_from_directory(path, target_size=target_size, class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)
    return batches

## Defining custom flow_from_directory generator for labels

In [186]:
def flow_from_directory_va(flow_from_directory_gen, get_data):
    for x in flow_from_directory_gen:
        idx = (flow_from_directory_gen.batch_index - 1) * flow_from_directory_gen.batch_size
        filenames = flow_from_directory_gen.filenames[idx : idx + flow_from_directory_gen.batch_size]
        labels = np.array([get_va_from_image[f] for f in filenames])
        yield x, labels

### VGG16

#### Loading batches, making destination folders

In [151]:
batches_flow_from_dir = get_data(folder_train_data)

Found 38332 images belonging to 74 classes.


In [152]:
list_filenames_affectnet = bt.filenames
print("total number of files are: ", len(list_filenames_affectnet))

total number of files are:  38332


In [44]:
folder_affectnet_conv_features_vgg16 = folder_affectnet + "features/vgg16/conv_features"
if not os.path.exists(folder_affectnet_conv_features_vgg16):
    os.makedirs(folder_affectnet_conv_features_vgg16)

#### Loading a pretrained model

Loading VGG16 model

In [46]:
model_vgg16 = applications.VGG16(weights='imagenet', include_top=True)
print ('VGG16 Model loaded')

VGG16 Model loaded


Splitting the layers, and making a Conv model

In [47]:
layers = model_vgg16.layers
layer_idx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]
conv_layers, fc_layers = layers[:layer_idx+1], layers[layer_idx+1:]

In [48]:
model_vgg16_conv = Sequential(conv_layers)

In [49]:
model_vgg16_conv.layers

##### temporary model with 2 neurons in the last layer

In [67]:
model_temp = applications.VGG16(weights='imagenet', include_top=True)
model_temp.layers.pop()

In [68]:
type(model_temp)

keras.engine.training.Model

In [69]:
new_model = Sequential()

In [70]:
new_model.add(model_temp)

In [71]:
new_model.add(Dense(2))

In [75]:
new_model.compile(loss='mse',optimizer='adam')

In [76]:
new_model.layers

In [110]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1000)              134260544 
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2002      
Total params: 134,262,546
Trainable params: 134,262,546
Non-trainable params: 0
_________________________________________________________________


### Training of the model

In [187]:
gen_va = flow_from_directory_va(batches_flow_from_dir, get_data)

In [188]:
gen_va

<generator object flow_from_directory_va at 0x7f2ef73c52b0>

In [189]:
new_model.fit_generator(gen_va, steps_per_epoch=20)

Epoch 1/1


TypeError: 'function' object is not subscriptable

In [137]:
new_model.summary(ImageDataGenerator())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1000)              134260544 
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2002      
Total params: 134,262,546
Trainable params: 134,262,546
Non-trainable params: 0
_________________________________________________________________
